In [2]:
!nvidia-smi

Fri Oct 15 12:40:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    33W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !mkdir /content/drive/MyDrive/EVA6/Capstone/dataset

In [ ]:
# !unzip /content/drive/MyDrive/EVA6/Capstone/construction_materials_dataset.zip -d /content/drive/MyDrive/EVA6/Capstone/dataset/

In [ ]:
# !git clone https://github.com/facebookresearch/detr.git

In [ ]:
# !rm -rf /content/EVA6_Capstone

In [4]:
!git clone https://github.com/gokul-pv/EVA6_Capstone

Cloning into 'EVA6_Capstone'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 39 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (39/39), done.


In [5]:
! pip install git+https://github.com/cocodataset/panopticapi.git

  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-kme4aofz
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-kme4aofz
  Created wheel for panopticapi: filename=panopticapi-0.1-py3-none-any.whl size=8306 sha256=138314353f1132542a071b736cea983571cbdb0f96a122c5255b625980d07c14
  Stored in directory: /tmp/pip-ephem-wheel-cache-e1jmz0ff/wheels/ad/89/b8/b66cce9246af3d71d65d72c85ab993fd28e7578e1b0ed197f1
Successfully built panopticapi


In [2]:
import os
import sys
import numpy as np
import torch
print(torch.__version__)

1.9.0+cu111


In [4]:
# sys.path.append(os.path.join(os.getcwd(), "detr/"))
sys.path.append(os.path.join(os.getcwd(), "/content/EVA6_Capstone/"))
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/EVA6_Capstone/']


In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
use_cuda, device

(True, device(type='cuda'))

In [6]:
import torchvision.transforms as T

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
# Load detr model
model, postprocessor = torch.hub.load('facebookresearch/detr', 'detr_resnet101_panoptic', pretrained=True, return_postprocessor=True, num_classes=250)
# Convert to eval mode
model = model.to(device)
model.eval()

print("Model Loaded")

Using cache found in /root/.cache/torch/hub/facebookresearch_detr_master


Model Loaded


In [11]:
data_path = [
      '/content/drive/MyDrive/EVA6/Capstone/dataset/aac_blocks'
]

In [12]:
def convert(o):
    if isinstance(o, np.generic): return o.item()  
    raise TypeError

In [13]:
from DatasetCreation import COCO_CATEGORIES, CUSTOM_CATEGORIES, COCO_NAMES, MAPPINGS, INFO, LICENSES, cat2id

In [14]:
ROOT_DIR = '/content/drive/MyDrive/EVA6/Capstone/dataset'

processing_file = ""
processing_data = []

image_id = 1
annotation_id = 1
segment_id = 1

GLOBAL_COCO = {
    "licenses": LICENSES,
    "info": INFO,
    "categories": CUSTOM_CATEGORIES,
    "annotations": [],
    "images": []
}

GLOBAL_PANOPTIC = {
    "licenses": LICENSES,
    "info": INFO,
    "categories": CUSTOM_CATEGORIES,
    "annotations": [],
    "images": []
}

In [15]:
import datetime
import time
import json
import traceback

In [16]:
from PIL import Image, ImageDraw, ImageFont
from panopticapi.utils import id2rgb, rgb2id
import io

In [ ]:
############################ Create DATASET ################################

# run through all folders in dataset
for category_path in data_path:
    # store starting time
    start = time.time()
    # get category name
    category_name = category_path.split("/")[7]
    print("Processing Category:", category_name)

    # open category coco file
    with open(os.path.join(category_path, "coco.json"), "r") as coco_file:
        category_coco = json.load(coco_file)
        
    images_root = os.path.join(category_path, 'images')
    
    TEMP_COCO_IMAGES = {}
    
    # Run over all images
    for im in category_coco["images"]:
        im['annotations'] = []
        TEMP_COCO_IMAGES[im['id']] = im
        
    for ann in category_coco["annotations"]:
        TEMP_COCO_IMAGES[ann['image_id']]["annotations"].append(ann)

      
    for i, image_coco in TEMP_COCO_IMAGES.items():
        # get image path
        ## This data can be used further for logging if failed while processing
        processing_file = os.path.join(images_root, image_coco['file_name'])
        processing_data = image_coco
        output_file_name = category_name + "_" + str(image_id) + ".jpg"
        output_file_path = os.path.join(ROOT_DIR, "images", output_file_name)
        
        output_mask_name = category_name + "_" + str(image_id) + ".png"
        output_mask_path = os.path.join(ROOT_DIR, "masks", output_mask_name)

        try:

          # Read this image and get shape of image
          imo = Image.open(processing_file).convert('RGB')

          try:
              h, w, c = np.array(imo).shape
          except:
              h, w = np.array(imo).shape
              c = 1

          # if no of channels != 3, open the image and convert it to 3 channel - RGB
          if c == 4 or c == 1:
              imo = imo.convert('RGB')
              h, w, c = np.array(imo).shape

          # Create a copy of image this will be used for further processing
          im = imo.copy()

          # Apply transform and convert image to batch
          # mean-std normalize the input image (batch-size: 1)
          img = transform(im).unsqueeze(0).to(device)  # [h, w, c] -> [1, c, ht, wt]

          # Generate output for image
          out = model(img)

          # Generate score
          # compute the scores, excluding the "no-object" class (the last one)
          scores = out["pred_logits"].softmax(-1)[..., :-1].max(-1)[0]

          # threshold the confidence
          keep = scores > 0.85

          # Keep only ones above threshold
          pred_logits, pred_boxes = out["pred_logits"][keep][:, :len(COCO_NAMES) - 1], out["pred_boxes"][keep] 
  
          # the post-processor expects as input the target size of the predictions (which we set here to the image size)
          result = postprocessor(out, torch.as_tensor(img.shape[-2:]).unsqueeze(0))[0]
          
          # The segmentation is stored in a special-format png
          panoptic_seg = Image.open(io.BytesIO(result['png_string'])).resize((w, h), Image.NEAREST)
          # (wp, hp) = panoptic_seg.size
          panoptic_seg = np.array(panoptic_seg, dtype=np.uint8).copy()   

          # We retrieve the ids corresponding to each mask
          panoptic_seg_id = rgb2id(panoptic_seg)

          # Merge predicted annotations      
          # 1. Get Mapping from predicted id to new ids

          unique_category_id = []
          for i, segment in enumerate(result['segments_info']):
              result['segments_info'][i]["category_id"] = MAPPINGS[result['segments_info'][i]["category_id"]]
              if result['segments_info'][i]["category_id"] not in unique_category_id:
                  unique_category_id.append(result['segments_info'][i]["category_id"])

          # Sort array
          unique_category_id.sort()
          
          unique_category_id_to_id =  {category_id: i for i, category_id in enumerate(unique_category_id)}
          unique_id_to_category_id =  {i: category_id for category_id, i in unique_category_id_to_id.items()}
          
          for i, segment in enumerate(result['segments_info']):
              result['segments_info'][i]["new_id"] = unique_category_id_to_id[result['segments_info'][i]["category_id"]]
          
          # Update original panoptic_seg_id array with new ids as the new segmentation combines different categories.
          custom_panoptic_seg_id = np.zeros((panoptic_seg_id.shape[0], panoptic_seg_id.shape[1]), dtype=np.uint8)
          
          # Update this custom panoptic seg matrix
          for i, segment in enumerate(result['segments_info']):
              custom_panoptic_seg_id[result['segments_info'][i]['id'] == panoptic_seg_id] = result['segments_info'][i]['new_id']
              
          # Create new Segmentation info
          # [{'area': 243, 'category_id': 3, 'id': 0, 'isthing': True},
          #   {'area': 730578, 'category_id': 184, 'id': 1, 'isthing': False}]
          
          custom_panoptic_segments_info = []
          for category_id in unique_category_id:
              custom_panoptic_segments_info.append({
                  'segment_id': unique_category_id_to_id[category_id], 
                  'category_id': category_id,
                  'bbox': [],
                  'area': 0,
                  'iscroud': 0,
                  'isthing': 0
              })

          # annotations of our construction things
          omask = processing_data['annotations']
          
          TEMP_ANNOTATIONS = []
          
          # Overlay things mask one at a time
          for annotation in omask:
              # overlay mask of construction things on top of detr output
              omask_image_id = get_overlayed_mask((h, w), annotation)
              custom_panoptic_seg_id[omask_image_id.astype(np.bool_)] = custom_panoptic_seg_id.max() + 1
              custom_panoptic_segments_info.append({
                  'segment_id': custom_panoptic_seg_id.max(), 
                  'category_id': cat2id[category_name], 
                  'bbox': annotation['bbox'],
                  'area': annotation['area'],
                  'iscroud': 0,
                  'isthing': 1
              })

              # append annotation of construction things in json file
              annotation["category_id"] = cat2id[category_name]
              annotation["image_id"] = image_id
              TEMP_ANNOTATIONS.append(annotation)
            
          # Convert to binary segment
          binary_masks = np.zeros((
              custom_panoptic_seg_id.max() + 1,
              custom_panoptic_seg_id.shape[0],
              custom_panoptic_seg_id.shape[1]),
              dtype=np.uint8
          )
              
          # for each binary mask, detect contours and create annotation for those contours
          if len(unique_category_id):
              # Skip the onse which are added by us
              for category_id in unique_category_id:
                  binary_masks[unique_category_id_to_id[category_id], :, :] = custom_panoptic_seg_id == unique_category_id_to_id[category_id]
                  annotation_info = coco_main(binary_masks[unique_category_id_to_id[category_id]], None, image_id, category_id, unique_category_id_to_id[category_id], 0)
                  if annotation_info is not None:
                      annotation_info["image_id"] = image_id
                      annotation_info["category_id"] = category_id
                      TEMP_ANNOTATIONS.append(annotation_info)
                      
                      custom_panoptic_segments_info[unique_category_id_to_id[category_id]]['bbox'] = annotation_info['bbox']
                      custom_panoptic_segments_info[unique_category_id_to_id[category_id]]['area'] = annotation_info['area']
          else:
              # Do something for the once where there are no predictions
              ## Probably mark them as None
              pass

          # Write data to global json and save files to image dir's
          
          # save image to new path as .jpg
          imo.save(output_file_path)
          
          # save panoptic image
          Image.fromarray(id2rgb(custom_panoptic_seg_id), 'RGB').save(output_mask_path)

          # create image_info object and append it to original list
          image_info = create_image_info(image_id, output_file_name, imo.size)
          image_info["original_file"] = processing_file

          GLOBAL_COCO["images"].append(image_info)
          GLOBAL_PANOPTIC["images"].append(image_info)

          for annotation in TEMP_ANNOTATIONS:
              annotation["id"] = annotation_id
              GLOBAL_COCO["annotations"].append(annotation)
              annotation_id += 1
              
          for segment_info in custom_panoptic_segments_info:
              segment_info["id"] = segment_id
              segment_id += 1
              
          GLOBAL_PANOPTIC["annotations"].append({
              "segments_info": custom_panoptic_segments_info,
              "file_name": output_mask_name,
              "image_id": image_id
          })

          # increment the image_count
          image_id += 1

        except Exception as e:
            # if there is any error, add info about it in errros file and procees to next image
            print("Error occurred while processig file:", processing_file)
            
            with open(os.path.join(ROOT_DIR, "error.json"), 'r') as error_file:
                error_json = json.load(error_file)
                
            with open(os.path.join(ROOT_DIR, "error.json"), 'w') as error_file:
                error_json["error"].append({
                    "processing_file": processing_file,
                    "processing_data": processing_data
                })
                
                json.dump(error_json, error_file)
                
            traceback.print_exc()
        del img   
        torch.cuda.empty_cache() 
        # break
    total_time_str = str(datetime.timedelta(seconds=int(time.time() - start)))
    print(f"Completed Category: {category_name}, Time Taken: {total_time_str}")

    # open the final json, and commit changes in that file
    with open(os.path.join(ROOT_DIR, "coco.json"), 'w') as output_json_file:
        json.dump(GLOBAL_COCO, output_json_file)
        
    with open(os.path.join(ROOT_DIR, "panoptic.json"), 'w') as output_json_file:
        json.dump(GLOBAL_PANOPTIC, output_json_file, default=convert)
        
    print(image_id, annotation_id, segment_id)
         
         

In [29]:
!rm -rf /content/drive/MyDrive/EVA6/Capstone/dataset/images/*
!rm -rf /content/drive/MyDrive/EVA6/Capstone/dataset/masks/*
!rm -rf /content/drive/MyDrive/EVA6/Capstone/dataset/coco.json
!rm -rf /content/drive/MyDrive/EVA6/Capstone/dataset/panoptic.json

In [18]:
from math import floor
import cv2
def get_overlayed_mask(image_size, annotations):
    height, width = image_size
    # # create a single channel height, width pixel black image
    blank_image = np.zeros((height, width))

    ## Show Code image
    # plt.imshow(image)
    # plt.show()

    # Create list of polygons to be drawn
    # for i, annotation in enumerate(annotations):
    polygons_list = []
    # Add the polygon segmentation
    for segmentation_points in annotations["segmentation"]:
        segmentation_points = np.multiply(segmentation_points, 1).astype(int)
        polygons_list.append(segmentation_points)

    for x in polygons_list:
        end = []
        if len(x) % 2 != 0:
            print(x)
        for l in range(0, len(x), 2):
            coords = [floor(x[l]), floor(x[l + 1])]
            end.append(coords)
        contours = np.array(end)
        if end == []:
            continue
        cv2.fillPoly(blank_image, pts=[contours], color=(1, 1, 1))
        ## Plot final image
        # plt.imshow(image)
        # plt.show()
    return blank_image

In [19]:
def coco_main(binary_mask, image_size, image_id, class_id, segmentation_id, iscrowd):

    ## --------- Prepare coco format from binary masks -------

    # {
    #     "id": 1,
    #     "image_id": 1,
    #     "category_id": 1,
    #     "segmentation": [
    #         []
    #     ],
    #     "area": 368501.0,
    #     "bbox": [
    #         0.0,
    #         74.18,
    #         751.32,
    #         544.58
    #     ],
    #     "iscrowd": 0,
    #     "attributes": {
    #         "occluded": false
    #     }
    # },

    category_info = {"id": class_id, "is_crowd": iscrowd}

    annotation_info = create_annotation_info(
        segmentation_id, image_id, category_info, binary_mask, image_size, tolerance=2
    )

    return annotation_info

In [20]:
from pycocotools import mask
def create_annotation_info(
    annotation_id,
    image_id,
    category_info,
    binary_mask,
    image_size=None,
    tolerance=2,
    bounding_box=None,
):

    if image_size is not None:
        binary_mask = resize_binary_mask(binary_mask, image_size)

    binary_mask_encoded = mask.encode(np.asfortranarray(binary_mask.astype(np.uint8)))

    area = mask.area(binary_mask_encoded)
    if area < 1:
        return None

    if bounding_box is None:
        bounding_box = mask.toBbox(binary_mask_encoded)

    if category_info["is_crowd"]:
        is_crowd = 1
        segmentation = binary_mask_to_rle(binary_mask)
    else:
        is_crowd = 0
        segmentation = binary_mask_to_polygon(binary_mask, tolerance)
        if not segmentation:
            return None

    annotation_info = {
        "id": annotation_id,
        "image_id": image_id,
        "category_id": category_info["id"],
        "iscrowd": is_crowd,
        "area": area.tolist(),
        "bbox": bounding_box.tolist(),
        "segmentation": segmentation,
        "width": binary_mask.shape[1],
        "height": binary_mask.shape[0],
    }

    return annotation_info

In [21]:
from skimage import measure
def binary_mask_to_polygon(binary_mask, tolerance=0):
    """Converts a binary mask to COCO polygon representation

    Args:
        binary_mask: a 2D binary numpy array where '1's represent the object
        tolerance: Maximum distance from original points of polygon to approximated
            polygonal chain. If tolerance is 0, the original coordinate array is returned.

    """
    polygons = []
    # pad mask to close contours of shapes which start and end at an edge
    padded_binary_mask = np.pad(
        binary_mask, pad_width=1, mode="constant", constant_values=0
    )
    contours = measure.find_contours(padded_binary_mask, 0.5)
    contours = np.subtract(contours, 1)
    for contour in contours:
        contour = close_contour(contour)
        contour = measure.approximate_polygon(contour, tolerance)
        if len(contour) < 3:
            continue
        contour = np.flip(contour, axis=1)
        segmentation = contour.ravel().tolist()
        # after padding and subtracting 1 we may get -0.5 points in our segmentation
        segmentation = [0 if i < 0 else i for i in segmentation]
        polygons.append(segmentation)

    return polygons

In [22]:
def close_contour(contour):
    if not np.array_equal(contour[0], contour[-1]):
        contour = np.vstack((contour, contour[0]))
    return contour

In [23]:
def create_image_info(
    image_id,
    file_name,
    image_size,
    date_captured=datetime.datetime.utcnow().isoformat(" "),
    license_id=1,
    coco_url="",
    flickr_url="",
):

    image_info = {
        "id": image_id,
        "file_name": file_name,
        "width": image_size[0],
        "height": image_size[1],
        "date_captured": date_captured,
        "license": license_id,
    }

    return image_info

In [24]:
from itertools import groupby

def binary_mask_to_rle(binary_mask):
    rle = {"counts": [], "size": list(binary_mask.shape)}
    counts = rle.get("counts")
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order="F"))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))

    return rle

In [25]:
def resize_binary_mask(array, new_size):
    image = Image.fromarray(array.astype(np.uint8) * 255)
    image = image.resize(new_size)
    return np.asarray(image).astype(np.bool_)

In [26]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 0.2 GB
Cached:    0.2 GB


/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:375: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


In [9]:
torch.cuda.empty_cache()

In [31]:
print(torch.cuda.get_device_properties(device))

_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [38]:
torch.cuda.max_memory_allocated()/1024**3

14.062496185302734

In [39]:
!ps -aux|grep python

root          79  0.1  0.2 194924 60936 ?        Sl   12:11   0:04 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         327  0.0  0.0  18380  2964 ?        S    12:41   0:00 bash -c tail -n +0 -F "/root/.config/Google/DriveFS/Logs/drive_fs.txt" | python3 /opt/google/drive/drive-filter.py > "/root/.config/Google/DriveFS/Logs/timeouts.txt" 
root         329  0.0  0.0  31740  9672 ?        S    12:41   0:00 python3 /opt/google/drive/drive-filter.py
root         475  5.8 15.0 39700884 4014956 ?    Ssl  12:49   0:49 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-0b4ee356-0a5c-49cb-8d5e-a20eff40822e.json
root         495  0.3  0.0 128408 16072 ?        Sl   12:49   0:02 /usr/bin/python3 /usr/local/lib/python3.7/dist-packages/debugpy/adapter --for-server 45135 --host 127.0.0.1 --port 16437 --server-access-token 7d866b28f28ae758002c38acbb9c1a